
# Step 1 — Geolocate the cities 🗺️

This notebook fetches `(lat, lon)` for **35 French cities** using the **OpenWeather Geocoding API**, then writes `cities_geoloc.csv` with columns: `city, lat, lon`.

> ⚠️ You'll need an OpenWeather API key.
- Create one (free) on openweathermap.org
- Set it as an env var `OPENWEATHER_API_KEY` **or** paste directly in the cell below.


In [ ]:

# 🔧 Config — paste your API key here (or set env var OPENWEATHER_API_KEY)
import os

OPENWEATHER_API_KEY = os.getenv("OPENWEATHER_API_KEY") or "PASTE_YOUR_KEY_HERE"

if not OPENWEATHER_API_KEY or OPENWEATHER_API_KEY == "PASTE_YOUR_KEY_HERE":
    print("⚠️ Please set OPENWEATHER_API_KEY env var or paste your key in OPENWEATHER_API_KEY variable.")

In [ ]:

# ✅ Libraries
import requests
import pandas as pd
from time import sleep

# Helper: query OpenWeather Geocoding API
def geocode_city(city, country_code="FR", api_key=None, pause=0.4):
    """Return (lat, lon) for a given city using OpenWeather Geocoding API.
    API docs: https://openweathermap.org/api/geocoding-api
    """
    api_key = api_key or OPENWEATHER_API_KEY
    url = "https://api.openweathermap.org/geo/1.0/direct"
    params = {
        "q": f"{city},{country_code}",
        "limit": 1,
        "appid": api_key
    }
    r = requests.get(url, params=params, timeout=20)
    r.raise_for_status()
    data = r.json()
    # Basic politeness to avoid hammering the API
    sleep(pause)
    if not data:
        return None, None
    return data[0].get("lat"), data[0].get("lon")

## City list (35)

In [ ]:

cities = [
    "Paris","Marseille","Lyon","Toulouse","Nice","Nantes","Montpellier","Strasbourg","Bordeaux","Lille",
    "Rennes","Reims","Le Havre","Saint-Étienne","Toulon","Grenoble","Dijon","Angers","Nîmes","Villeurbanne",
    "Clermont-Ferrand","Le Mans","Aix-en-Provence","Brest","Tours","Amiens","Limoges","Perpignan","Metz",
    "Besançon","Orléans","Caen","Nancy","Rouen","Mulhouse"
]
len(cities)

## Geocode all cities

In [ ]:

rows = []
for c in cities:
    try:
        lat, lon = geocode_city(c)
        rows.append({"city": c, "lat": lat, "lon": lon})
        print(f"{c:20s} -> {lat}, {lon}")
    except Exception as e:
        print(f"Error for {c}: {e}")
        rows.append({"city": c, "lat": None, "lon": None})

df = pd.DataFrame(rows)
df

## Save to CSV

In [ ]:

out_path = "cities_geoloc.csv"
df.to_csv(out_path, index=False)
print(f"Saved -> {out_path}")